<a href="https://colab.research.google.com/github/Bharathkumar1404/LLM-Assignment/blob/main/22018647.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets

In [3]:
import pandas as pd
import numpy as np
import transformers
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
from datasets import load_dataset

R8_ds = load_dataset("dxgp/R8")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
type(R8_ds)

datasets.dataset_dict.DatasetDict

In [45]:
# Convert the dataset's train, test, or validation split to pandas DataFrame
train_R8 = R8_ds['train'].to_pandas()
test_R8 = R8_ds['test'].to_pandas()
val_R8 = R8_ds['validation'].to_pandas()

In [46]:
# Remove the 'Unnamed: 0' column
train_R8 = train_R8.drop(columns=['Unnamed: 0'])
test_R8 = test_R8.drop(columns=['Unnamed: 0'])
val_R8 = val_R8.drop(columns=['Unnamed: 0'])

print(train_R8.head(10))

                                                text  label
0  swedish prime minister s china visit boosts tr...      7
1  gaf gaf seeks all of borg warner bor gaf corp ...      0
2  mcclain industries inc mccl st qtr dec net shr...      2
3  foothill group fgi sees better first quarter d...      2
4  tenneco tgt buys unisys uis unit tenneco inc s...      0
5  carling o keefe ckb sells oil unit takes gain ...      0
6  reuters to buy i p sharp of canada reuters hol...      0
7  wardair international ltd year net shr dlrs vs...      2
8  borg warner bor to sell unit for mln dlrs borg...      0
9  nvhomes nvh sets two for one split nvhomes lp ...      2


In [36]:
print(train_R8.columns)

Index(['text', 'label'], dtype='object')


In [37]:
labels = train_R8['label'].unique()
print(labels)

[7 0 2 1 5 6 4 3]


acq: Acquisition-related news.
crude: News related to crude oil.
earn: Earnings reports and related news.
grain: News related to grain market activities.
interest: News about interest rates.
money-fx: News related to foreign exchange and money markets.
ship: News related to shipping.
trade: News related to international trade.

In [8]:
# count the label class into dictionary
labels = train_R8['label'].replace([0,1,2,3,4,5,6,7], ['acquistion', 'crude','earn','grain','interest','money-fx','ship','trade']).value_counts().to_dict()
print(train_R8)

      Unnamed: 0                                               text  label
0           5162  swedish prime minister s china visit boosts tr...      7
1           4594  gaf gaf seeks all of borg warner bor gaf corp ...      0
2           3876  mcclain industries inc mccl st qtr dec net shr...      2
3           4047  foothill group fgi sees better first quarter d...      2
4           4406  tenneco tgt buys unisys uis unit tenneco inc s...      0
...          ...                                                ...    ...
4931        3250  marcus mrcs votes pct stock dividend marcus co...      2
4932        1701  protocomdevices prcm sees more profit erosion ...      2
4933         229  franklin minnesota insured sets payout mthly d...      2
4934        1993  varity vat expects th qtr full year loss varit...      2
4935        3743  n z trade surplus mln dlrs in february prelimi...      7

[4936 rows x 3 columns]


In [9]:
import plotly.graph_objects as go

# Data for the bar chart
x = list(labels.keys())
y = list(labels.values())
# Create a bar chart
fig = go.Figure(data=go.Bar(x=x, y=y))

# Customize the layout
fig.update_layout(title='Bar Chart Example', xaxis_title='Labels', yaxis_title='Label count')

# Show the figure
fig.show()

# Create a pie chart
fig = go.Figure(data=[go.Pie(labels=x, values=y)])

# Customize the layout
fig.update_layout(title='Pie Chart Example')


# Show the figure
fig.show()



In [10]:
from transformers import BertTokenizer

# Load a pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [11]:
# function that tokenize the dataframe and converts it into tf dataset
def preprocess(data):
    """
    function that tokenize the dataframe and converts it into tf dataset

    Args
    data(dataframe): takes in a pandas dataframe

    Returns:
    tf.data.dataset
    """
    texts = data['text'].tolist() # convert text to list
    #texts = [s.decode('utf-8') for s in texts] # decode text into list
    labels = data['label'].tolist() # get labels

    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512,return_tensors='tf')

    dataset = tf.data.Dataset.from_tensor_slices((
        dict(encodings),
        labels
    )).batch(32)
    return dataset

In [12]:
# apply preprocess function to train_df
train_data = preprocess(train_R8)

# apply preprocess function to val_df
val_data = preprocess(val_R8)

# appply preprocess function to test_df
test_data = preprocess(test_R8)

In [13]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
# Load pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=8)  # Adjust num_labels based on your classification task


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss='sparse_categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' for integer labels
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5  # Adjust epochs based on your needs
)


Epoch 1/5
